## Lesson 2: Connecting with a CRM

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: To get a refund for your broken mug, please contact our customer
       support team directly. You can reach them by phone at
       1-800-123-4567 or by email at support@ourcompany.com. When you
       reach out, please be prepared to provide the following details:
       - When you purchased the mug - A description of the issue with
       the mug - Any order or purchase information you have  Our team
       will review your request and get back to you as soon as
       possible to process the refund. I apologize that I cannot
       directly assist you, but I hope you understand that I am
       limited in my abilities as an AI agent. Please let me know if
       you have any other questions!

Session ID: f34d0f8c-0f63-401b-a316-4eaa3f3a5858


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined in the video is at  <code>./ro_shared_data/functions/lambda_stage_1.py</code> </p>

In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': 'afd8b031-1e38-46a3-a014-b120b8f8e573',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 21 Oct 2024 11:25:28 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'afd8b031-1e38-46a3-a014-b120b8f8e573',
   'x-amz-apigw-id': 'f_4GWFvkPHcEY_A=',
   'x-amzn-trace-id': 'Root=1-67163a28-3fadb43c4e6e52f0316b5f61'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:767397763448:function:dlai-support-agent-Y7V97'},
  'actionGroupId': 'RDTWRVFEEX',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'TXEVAVOSGG',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 10, 21, 11, 25, 28, 376945, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is priva

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: The <REDACTED> function returned an error indicating that details are
       missing. I should ask the customer for more information before
       escalating the request.  I apologize, but I don't have enough
       details to fully process your refund request for the broken
       mug. Could you please provide some additional information, such
       as:  - When and where did you purchase the mug? - Do you have a
       receipt or order number? - Can you describe the issue with the
       mug in more detail?  With this additional information, I can
       better understand your situation and escalate your request to
       the support team. Please let me know if you can provide any of
       those details.

Session ID: 5bb216d7-1083-4408-9112-9b7a6541aa93


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  The customer has provided the same information as before, but has
  not given any additional details about the purchase or the issue
  with the mug. I still don't have enough information to fully process
  the refund request, so I will need to ask for more details again.

Observation:
  Type: FINISH

Final response:
  I apologize, but I still don't have enough details to fully process
  your refund request for the broken mug. Could you please provide
  some additional information, such as:  - When and where did you
  purchase the mug?  - Do you have a receipt or order number? - Can
  you describe the issue with the mug in more detail?  With this
  additional information, I can better understand your situation and
  escalate your request to the support team. Please let me know if you
  can provide any of those details.


Session ID: 5bb216d7-1083-4408-9112-9b7a6541aa93
